In [ ]:
import os
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.model_selection import train_test_split
from torchvision import transforms
from torch.utils.data import DataLoader, Dataset
from PIL import Image
import matplotlib.pyplot as plt
from pickle import dump, load

In [ ]:
# Step 1: Load and preprocess images
def load_images(images_folder, save_file_to=None):
    X_original = []
    X = []
    y = []

    for subdir, dirs, files in os.walk(images_folder):
        for file in files:
            if file.endswith(('jpg', 'jpeg', 'png')):
                img_path = os.path.join(subdir, file)
                label = os.path.basename(subdir)

                image = Image.open(img_path).convert('L')  # Convert to grayscale
                image = image.resize((48, 48))  # Resize to 48x48
                X_original.append(np.array(image).flatten())
                X.append(np.array(image).flatten())
                y.append(label)

    if save_file_to:
        with open(save_file_to, "wb") as f:
            dump((X_original, X, y), f, protocol=5)

    return np.array(X_original), np.array(X), np.array(y)

images_folder = "../images"
dataset_file = "dataset_dump.pkl"


X_original, X, y = load_images(images_folder, save_file_to=dataset_file)

In [ ]:
with open("dataset_dump.pkl", "rb") as f:
    X_original,X,y = load(f)
    
X
y

In [ ]:
# Step 2: PCA + StandardScaler preprocessing
def preprocessing_data(X, y, n_components=100, save_file_to=None):
    X = X / 255.0  # Normalize pixel values to [0, 1]

    label_encoder = LabelEncoder()
    y_encoded = label_encoder.fit_transform(y)

    pca = PCA(n_components=n_components)
    X_reduced = pca.fit_transform(X)

    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X_reduced)

    if save_file_to:
        with open(save_file_to, "wb") as f:
            dump((X_scaled, y_encoded, label_encoder, pca, scaler), f, protocol=5)

    return X_scaled, y_encoded, label_encoder, pca, scaler

X_scaled, y_encoded, label_encoder, pca, scaler = preprocessing_data(X, y, save_file_to="preprocessed_data.pkl")


In [ ]:
with open("labelencoder_standardscaler_pca_normalizers_dump.pkl", "rb") as f:
    X_scaled, y_encoded, label_encoder, pca, scaler = load(f)

In [ ]:
# Step 3: Dataset and Transformations
class CustomDataset(Dataset):
    def __init__(self, X, y, transform=None):
        self.X = X
        self.y = y
        self.transform = transform

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        data = self.X[idx].reshape(10, 10)  # Reshape PCA-reduced data (100 components -> 10x10 grid)
        label = self.y[idx]

        if self.transform:
            data = Image.fromarray((data * 255).astype(np.uint8))  # Convert to PIL for transforms
            data = self.transform(data)

        return data, label


In [ ]:
# Define transformations
image_transforms = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(15),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5], std=[0.5])
])

# Split data into train/test sets
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y_encoded, test_size=0.2, random_state=42)

# Create datasets and dataloaders
train_dataset = CustomDataset(X_train, y_train, transform=image_transforms)
test_dataset = CustomDataset(X_test, y_test, transform=image_transforms)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)


In [ ]:
# Step 4: Neural Network Model
class SimpleNN(nn.Module):
    def __init__(self):
        super(SimpleNN, self).__init__()
        self.fc1 = nn.Linear(100, 50)
        self.dropout = nn.Dropout(0.5)
        self.fc2 = nn.Linear(50, len(np.unique(y_encoded)))

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        return x

In [ ]:
# Step 5: Training
model = SimpleNN()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training loop
num_epochs = 10
for epoch in range(num_epochs):
    model.train()
    for batch_idx, (data, labels) in enumerate(train_loader):
        data = data.view(data.size(0), -1)  # Flatten data
        labels = labels.long()

        optimizer.zero_grad()
        outputs = model(data)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}")


In [ ]:
# Step 6: Evaluation
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for data, labels in test_loader:
        data = data.view(data.size(0), -1)
        labels = labels.long()
        outputs = model(data)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

accuracy = 100 * correct / total
print(f"Accuracy: {accuracy:.2f}%")


In [ ]:
# Step 7: Visualizing Predictions
def visualize_predictions(model, dataset, label_encoder):
    model.eval()
    with torch.no_grad():
        random_indices = np.random.randint(0, len(dataset), 5)
        for idx in random_indices:
            data, label = dataset[idx]
            data = data.view(1, -1)
            outputs = model(data)
            _, predicted = torch.max(outputs.data, 1)

            true_label = label_encoder.inverse_transform([label])[0]
            pred_label = label_encoder.inverse_transform(predicted.numpy())[0]

            # Reshape and visualize
            original_data = dataset[idx][0].view(10, 10).numpy()
            plt.imshow(original_data, cmap='gray')
            plt.title(f"True: {true_label}, Predicted: {pred_label}")
            plt.axis("off")
            plt.show()

visualize_predictions(model, test_dataset, label_encoder)